In [1]:
from trainmodel import *

Using TensorFlow backend.


In [2]:
file_name = "short.csv"

In [3]:
df = k.pd.read_csv(file_name,encoding='ISO-8859-1')
df = df[df.duration >=2]
filtered_df = getsplit.filter_df(df)
# filtered_df.head()

In [4]:
X_train, X_test, y_train, y_test = k.getsplit.split_people(filtered_df)

In [5]:
train_count = k.Counter(y_train)
test_count = k.Counter(y_test)
acc_to_beat = test_count.most_common(1)[0][1] / float(np.sum(list(test_count.values())))

In [6]:
y_train = k.to_categorical(y_train)
print(y_train.shape)

[2, 2, 3, 1, 3, 0, 1, 1, 2, 2, 3, 3, 2, 3, 3, 2, 3, 1, 2, 2, 2, 1, 0, 3, 1, 1, 0, 0, 1, 2, 2, 1, 0, 2, 2, 1, 2, 2, 1, 3, 0, 2, 0, 2, 1, 3, 3, 3, 3, 2, 1, 0, 1, 3, 0, 1, 3, 2, 1, 0, 0, 3, 1, 1, 1, 3, 3, 2, 2, 0, 2, 3, 2, 3, 0, 3, 2, 1, 1, 2, 1, 0, 1, 2, 1, 0, 0, 3, 0, 2, 3, 2, 1, 3, 0, 1, 2, 1, 1, 2, 0, 2, 0, 2, 0, 3, 2, 0, 3, 1, 0, 0, 3, 1, 1, 0, 0, 2, 3, 2, 0, 0, 2, 3, 2, 1, 0, 0, 3, 3, 3, 3, 1, 2, 2, 3, 1, 0, 3, 1, 2, 3, 0, 0, 3, 2, 2, 3, 2, 2, 2, 2, 1, 2, 1, 1, 3, 1, 2, 0, 1, 0, 1, 3, 2, 0, 1, 0, 1, 1, 2, 3, 2, 3, 0, 3, 2, 1, 0, 0, 3, 0, 1, 1, 3, 1, 3, 1, 3, 2, 3, 1, 3, 3, 1, 2, 2, 0, 0, 1, 2, 0, 0, 3, 3, 2, 0, 0, 2, 0, 3, 0, 1, 2, 3, 3, 1, 0, 2, 3, 0, 1, 0, 1, 2, 0, 2, 2, 2, 0, 1, 1, 0, 3, 0, 0, 3, 3, 1, 3, 3, 0, 3, 2, 2, 0, 1, 1, 1, 3, 2, 2, 2, 2, 1, 1, 1, 0, 0, 2, 1, 1, 1, 0, 3, 3, 3, 0, 0]
(269, 4)


In [7]:
y_test = k.to_categorical(y_test)
print(y_test.shape)

[1, 0, 3, 0, 3, 1, 0, 0, 3, 0, 1, 0, 3, 0, 3, 3, 0, 2, 2, 1, 0, 3, 3, 2, 1, 0, 1, 2, 1, 2, 2, 0, 0, 1, 1, 2, 1, 1, 3, 3, 0, 0, 2, 0, 3, 1, 2, 0]
(48, 4)


In [8]:
import multiprocess as mp

In [9]:
print("Loading Wav file")
pool = mp.Pool(processes=mp.cpu_count())
X_train = pool.map(k.get_wav, X_train)
X_test = pool.map(k.get_wav, X_test)

Loading Wav file


In [10]:
print('Converting to MFCC....')
X_train = pool.map(k.to_mfcc, X_train)
X_test = pool.map(k.to_mfcc, X_test)

Converting to MFCC....


In [11]:
# Create segments from MFCCs
X_train, y_train = k.make_segments(X_train, y_train)
X_validation, y_validation = k.make_segments(X_test, y_test)

In [12]:
X_train, _, y_train, _ = train_test_split(X_train, y_train, test_size=0.1)

In [18]:
csvfile='output.csv'
results = []
acc = []

In [36]:
def alter_model(X_train,y_train,X_validation,y_validation, batch_size=128):
    rows = X_train[0].shape[0]
    cols = X_train[0].shape[1]
    val_rows = X_validation[0].shape[0]
    val_cols = X_validation[0].shape[1]
    num_classes = len(y_train[0])
    # input image dimensions to feed into 2D ConvNet Input layer
    input_shape = (rows, cols, 1)
    X_train = X_train.reshape(X_train.shape[0], rows, cols, 1 )
    X_validation = X_validation.reshape(X_validation.shape[0],val_rows,val_cols,1)
    device_name = "/device:GPU:1"
    # device_name = "/cpu:0"
    
    window_size = 3
    EPOCHS = 20
    dropout_rate = .05
    
    with tf.device(device_name):
        print("Using: {0}".format(device_name))
        print('X_train shape:', X_train.shape)
        print(X_train.shape[0], 'training samples')

        model = Sequential()
        # Layer  1
        model.add(Conv2D(16, 
                        kernel_size=(window_size,window_size),
                        activation='relu',
                        
                        input_shape=input_shape))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(window_size - 1, window_size - 1)))
        
        # Layer 2
        model.add(Conv2D(32,
                        kernel_size=(window_size,window_size), 
                        activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(window_size -1, window_size - 1)))
        model.add(Dropout(dropout_rate))

        
        model.add(Flatten())
        
        # Layer 3
        model.add(Dense(256, activation='relu'))
        model.add(Dropout(dropout_rate))

        # Layer 4
        model.add(Dense(num_classes, activation='softmax'))
        model.add(Dropout(dropout_rate))
        
        
        # Evaluate the Layer
        model.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])

        # Stops training if accuracy does not change at least 0.005 over 10 epochs
        es = EarlyStopping(monitor='acc', 
                           min_delta=.005,
                           patience=10, 
                           verbose=1, 
                           mode='auto')

        # Creates log file for graphical interpretation using TensorBoard
        tb = TensorBoard(log_dir='./logs', 
                         histogram_freq=0, 
                         batch_size=32, 
                         write_graph=True, 
                         write_grads=True,
                         write_images=True, 
                         embeddings_freq=0, 
                         embeddings_layer_names=None,
                         embeddings_metadata=None)

        # Image shifting
        datagen = ImageDataGenerator(width_shift_range=0.05)

        # Fit model using ImageDataGenerator
        csv_logger = CSVLogger('train-validation.csv', 
                               append=True, 
                               separator=',')
        
        model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size),
                            steps_per_epoch=len(X_train)/32, 
                            epochs=EPOCHS,
                            callbacks=[
                                es,
                                tb,
                                csv_logger
                            ], 
                            validation_data=(X_validation,y_validation))

        return (model)

In [37]:
for i in range (0,1):
    model = alter_model(np.array(X_train), 
                        np.array(y_train), 
                        np.array(X_validation),
                        np.array(y_validation))
    y_predicted = accuracy.predict_class_all(k.create_segmented_mfccs(X_test), model)
    print('Training samples:', train_count)
    print('Testing samples:', test_count)
    print('Accuracy to beat:', acc_to_beat)
    print('Confusion matrix of total samples:\n', np.sum(accuracy.confusion_matrix(y_predicted, y_test),axis=1))
    print('Confusion matrix:\n',accuracy.confusion_matrix(y_predicted, y_test))
    print('Accuracy:', accuracy.get_accuracy(y_predicted,y_test))

    results.append(accuracy.confusion_matrix(y_predicted, y_test))
    acc.append(accuracy.get_accuracy(y_predicted,y_test))
    with open(csvfile, 'a') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerow([i,accuracy.get_accuracy(y_predicted,y_test)])
        
    model.summary()


Using: /device:GPU:1
X_train shape: (15494, 13, 30, 1)
15494 training samples
Epoch 1/20
485/484 [==============================] - 9s 18ms/step - loss: 1.3135 - acc: 0.4986 - val_loss: 1.2339 - val_acc: 0.4998
Epoch 2/20
485/484 [==============================] - 9s 19ms/step - loss: 0.8376 - acc: 0.6738 - val_loss: 1.5728 - val_acc: 0.4497
Epoch 3/20
485/484 [==============================] - 10s 20ms/step - loss: 0.6204 - acc: 0.7555 - val_loss: 1.8812 - val_acc: 0.4852
Epoch 4/20
485/484 [==============================] - 9s 19ms/step - loss: 0.5182 - acc: 0.7967 - val_loss: 1.7505 - val_acc: 0.5226
Epoch 5/20
485/484 [==============================] - 9s 19ms/step - loss: 0.4569 - acc: 0.8229 - val_loss: 1.9960 - val_acc: 0.5109
Epoch 6/20
485/484 [==============================] - 9s 19ms/step - loss: 0.4130 - acc: 0.8424 - val_loss: 2.2517 - val_acc: 0.5086
Epoch 7/20
485/484 [==============================] - 10s 20ms/step - loss: 0.3709 - acc: 0.8598 - val_loss: 1.9697 - val_a